# README

The goal in this script is to simplify generating eoir synthetic data.  
An interval of time is specified and the data is generated over that  
period.  

Before running or if using a different scenario, the user needs to  
do the following:
* Open the STK scenario 
* Set the animation time & time step

In [1]:
# Replace X with your version #. The python api must be installed prior to running this code
# pip install "C:\Program Files\AGI\STK 12\bin\AgPythonAPI\agi.stk12-12.X.0-py3-none-any.whl"

import os 
from datetime import timedelta
from datetime import datetime
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# This section gets the IAgStkObjectRoot interface 
stk      = STKDesktop.AttachToApplication()  
stk.UserControl = True
root     = stk.Root                           
scenario = root.CurrentScenario 


def findObjects():
    # If the scenario has multiple objects with sensors, sweep through them all.
    # We want to check if the objects have EOIR sensors & ignore other payloads.
    parClass = []
    parName  = []
    chlClass = []
    chlName  = []  
    if scenario.HasChildren == True:
        for i, parObj in enumerate(scenario.Children):
                for j, type in enumerate(parObj.Children):
                    if parObj.HasChildren == True and parObj.Children.Item(j).ClassName == 'Sensor': 
                        if parObj.Children.Item(j).PatternType.name == 'eSnEOIR':
                            parClass.append(parObj.ClassName)
                            parName.append(parObj.InstanceName)
                            chlClass.append(parObj.Children.Item(j).ClassName)
                            chlName.append(parObj.Children.Item(j).InstanceName)
                        else:
                            pass
                    else:
                        pass
    else:
        print("Scenario is empty.")
    return parClass,parName,chlClass,chlName

def createfolders(filepath,sensor):
    # Generate folders for the data to go 
    # Directory structure is based on sensor name
    newdir = os.mkdir("{}output/".format(filepath))
    for (i) in sensor: 
        #print("{}output/{}".format(filepath,i))
        os.mkdir("{}output/{}".format(filepath,i))

In [9]:
def eoirdata(parClass,parName,chlClass,sensor,filepath,z):

    # Use connect commands to generate the EOIR Syn Scene &/or Txt file 
    print('Generating files....')
            
    for k in range(z):
        for j in range(len(chlClass)):    
            print(parClass[j],parName[j],chlClass[j],sensor[j],
            filepath,sensor[j],str(int(root.CurrentTime)).zfill(9),str(k).zfill(4))
            print("EOIRDetails */{}/{}/{}/{} SaveSceneImage "\
            "\"{}output/{}/image{}_{}.bmp\""
            .format(parClass[j],parName[j],chlClass[j],sensor[j],
            filepath,sensor[j],str(round(float(root.CurrentTime),1)).zfill(9),str(k).zfill(4)))
                
            print("EOIRDetails */{}/{}/{}/{} SaveSceneRawData "\
            "\"{}output/{}/text{}_{}.txt\""
            .format(parClass[j],parName[j],chlClass[j],sensor[j],
            filepath,sensor[j],str(round(float(root.CurrentTime),1)).zfill(9),str(k).zfill(4)))
                
            root.ExecuteCommand("EOIRDetails */{}/{}/{}/{} SaveSceneImage "\
            "\"{}output/{}/image{}_{}.bmp\""
            .format(parClass[j],parName[j],chlClass[j],sensor[j],
            filepath,sensor[j],str(round(float(root.CurrentTime),1)).zfill(9),str(k).zfill(4)))
                
            root.ExecuteCommand("EOIRDetails */{}/{}/{}/{} SaveSceneRawData "\
            "\"{}output/{}/text{}_{}.txt\""
            .format(parClass[j],parName[j],chlClass[j],sensor[j],
            filepath,sensor[j],str(round(float(root.CurrentTime),1)).zfill(9),str(k).zfill(4)))
            
            root.StepForward()  
        else: 
            root.Pause    
    
    print('Complete.')

 


In [ ]:
#Remove this section as it should be it's own separate action. 
#Not all sensors have taret objects, so this report will be a challenge. 
# def eoirReport(parClass,parName,chlClass,chlName,mdlfile,filepath):
    
#     reportName = 'EOIR Sensor-to-Target Metrics'
#     #there's only one sensor in this scenario, down the line turn this into a loop
#     sensorPath = '*/{}/{}/{}/{}'.format(parClass[0],parName[0],chlClass[0],chlName[0])
#     sensor     = root.GetObjectFromPath(sensorPath)
#     exportFile = os.path.join(filepath, mdlfile + "_STTM_Report.txt")          
                                 
#     command = "ReportCreate {} Type Export Style \"{}\" File \"{}\" "\
#               "TimePeriod \"{}\" \"{}\" Timestep 0.1 "\
#               "AdditionalData \"Satellite/Satellite1 Band1".format(
#                 sensorPath,
#                 reportName,
#                 exportFile,
#                 scenario.StartTime,
#                 scenario.StopTime)
    
#     print("Exporting {} report...".format(reportName))
#     print(command)
#     #root.ExecuteCommand(command)

In [11]:
def main():

    # File path for the open scenario, close any STK scenarios 
    # or background AGgUiApplication processes 
    filepath = root.ExecuteCommand('GetDirectory / Scenario').Item(0)
    
    ######### vvvvvv USER DEFINED vvvvvv ######### 
    # set the scenario animation properties
    animation               = scenario.Animation
    # set the animation time 
    animation.TimePeriod.UseAnalysisStartTime = False
    animation.TimePeriod.UseAnalysisStopTime = False
    animation.StartTime     = '24 Oct 2022 21:03:51.130' #'DD Mmm YYYY 00:00:00.000' 
    animation.AnimCycleTime = '24 Oct 2022 21:41:16.407' #'DD Mmm YYYY 00:00:00.000'
    
    
    animation.EnableAnimCycleTime = True
    animation.AnimCycleType = AgEScEndLoopType.eEndTime
    # animation.AnimCycleType = AgEScEndLoopType.eLoopAtTime 
    animation.AnimStepValue = 60 # Change this to set the interval for the frames/raw data
    
    ############################################## 
    
    # Total frames (k) generated based on scenario animation time/timestep
    d1 = datetime.strptime(animation.StartTime,     "%d %b %Y %H:%M:%S.%f")
    d2 = datetime.strptime(animation.AnimCycleTime,      "%d %b %Y %H:%M:%S.%f")
    k  = int(timedelta.total_seconds(d2 - d1)/animation.AnimStepValue)
    #k  = 4
    print("Number of Iterations: {}".format(k)) 

    # Identify all parent objects with EOIR sensors 
    parClass,parName,chlClass,chlName  = findObjects()
    print(parClass, parName, chlClass, chlName)
    # Check if the save to directories exist 
    # Create them before generating data
    if os.path.exists("{}output/".format(filepath)) == False:
        print('Creating Directories...')
        createfolders(filepath,chlName)
    else:
        print('Directories Exist')
    
    root.Rewind()
    
    # Animate and generate data for each timestep 
    # print(parClass,parName,chlClass,chlName,filepath,k)
    eoirdata(parClass,parName,chlClass,chlName,filepath,k)
    
if __name__ == '__main__':
    main()

Number of Iterations: 37
['Aircraft'] ['Sensor'] ['Sensor'] ['EOIRsensor']
Directories Exist
Generating files....
Aircraft Sensor Sensor EOIRsensor C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\ EOIRsensor 000001914 0000
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneImage "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/image0001914.7_0000.bmp"
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneRawData "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/text0001914.7_0000.txt"
Aircraft Sensor Sensor EOIRsensor C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\ EOIRsensor 000001974 0001
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneImage "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/im

Aircraft Sensor Sensor EOIRsensor C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\ EOIRsensor 000002814 0015
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneImage "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/image0002814.7_0015.bmp"
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneRawData "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/text0002814.7_0015.txt"
Aircraft Sensor Sensor EOIRsensor C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\ EOIRsensor 000002874 0016
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneImage "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/image0002874.7_0016.bmp"
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneRawData "C:\Users\nkazmi\OneDrive 

Aircraft Sensor Sensor EOIRsensor C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\ EOIRsensor 000003714 0030
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneImage "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/image0003714.7_0030.bmp"
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneRawData "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/text0003714.7_0030.txt"
Aircraft Sensor Sensor EOIRsensor C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\ EOIRsensor 000003774 0031
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneImage "C:\Users\nkazmi\OneDrive - ANSYS, Inc\Documents\SSRs\AFTC Tangram\AFTC_Setup2_Nova_amto_test\output/EOIRsensor/image0003774.7_0031.bmp"
EOIRDetails */Aircraft/Sensor/Sensor/EOIRsensor SaveSceneRawData "C:\Users\nkazmi\OneDrive 